In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = {'func': 'getbuyersguide',
                        'payload': '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[a, 'Type_Code'] + '","partkey":"' + input_.loc[a, 'Part_Key'] + '"}}}',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    pass
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for tr in list_tr:
                    list_td = [text.strip() for text in tr.xpath('./td/text()')]
                    list_vehicle.append(' '.join(list_td).strip())

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_TCPK': input_.loc[a, 'JOIN_TCPK'],
                                         'Vehcile': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Type_Code': input_.loc[a, 'Type_Code'],
                                     'Part_Key': input_.loc[a, 'Part_Key'],
                                     'JOIN_TCPK': input_.loc[a, 'JOIN_TCPK']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Type_Code'] + ' - ' + input_.loc[a, 'Part_Key'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_TCPK'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Type_Code', 'Part_Key'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3110

[ok] - 2308 - 8165540
[尝试次数：1] - [剩余数量：3098] - [当前时间：13:35:44]

[ok] - 2308 - 5548860
[尝试次数：1] - [剩余数量：3097] - [当前时间：13:35:44]

[ok] - 2308 - 8165524
[尝试次数：1] - [剩余数量：3096] - [当前时间：13:35:44]

[ok] - 2308 - 5548866
[尝试次数：1] - [剩余数量：3095] - [当前时间：13:35:44]

[ok] - 2308 - 5548863
[尝试次数：1] - [剩余数量：3094] - [当前时间：13:35:44]

[ok] - 2308 - 8165516
[尝试次数：1] - [剩余数量：3093] - [当前时间：13:35:44]

[ok] - 2308 - 4672026
[尝试次数：1] - [剩余数量：3092] - [当前时间：13:35:44]

[ok] - 2308 - 8165544
[尝试次数：1] - [剩余数量：3091] - [当前时间：13:35:44]

[ok] - 2308 - 3886863
[尝试次数：1] - [剩余数量：3090] - [当前时间：13:35:45]

[ok] - 2308 - 3886971
[尝试次数：1] - [剩余数量：3089] - [当前时间：13:35:45]

[ok] - 2308 - 8321732
[尝试次数：1] - [剩余数量：3088] - [当前时间：13:35:45]

[ok] - 2308 - 8165552
[尝试次数：1] - [剩余数量：3087] - [当前时间：13:35:45]

[ok] - 2308 - 8020564
[尝试次数：1] - [剩余数量：3086] - [当前时间：13:35:45]

[ok] - 2308 - 8165548
[尝试次数：1] - [剩余数量：3085] - [当前时间：13:35:45]

[ok] - 2308 - 8165560
[尝试次数：1] - [剩余数量：3084] - [当前时间：13:35:45]

[ok] - 2308 - 8165556
[尝试次数：1]

[ok] - 2308 - 8165696
[尝试次数：5] - [剩余数量：2970] - [当前时间：13:36:18]

[ok] - 2308 - 8165668
[尝试次数：16] - [剩余数量：2969] - [当前时间：13:36:18]

[ok] - 2308 - 8165712
[尝试次数：4] - [剩余数量：2968] - [当前时间：13:36:18]

[ok] - 2308 - 8165660
[尝试次数：21] - [剩余数量：2967] - [当前时间：13:36:18]

[ok] - 2308 - 8165708
[尝试次数：4] - [剩余数量：2966] - [当前时间：13:36:18]

[ok] - 2308 - 8165716
[尝试次数：5] - [剩余数量：2965] - [当前时间：13:36:19]

[ok] - 2308 - 8165700
[尝试次数：8] - [剩余数量：2964] - [当前时间：13:36:19]

[ok] - 2308 - 8165732
[尝试次数：2] - [剩余数量：2963] - [当前时间：13:36:19]

[ok] - 2308 - 8165704
[尝试次数：8] - [剩余数量：2962] - [当前时间：13:36:19]

[ok] - 2308 - 8165736
[尝试次数：3] - [剩余数量：2961] - [当前时间：13:36:20]

[ok] - 2308 - 8165724
[尝试次数：7] - [剩余数量：2960] - [当前时间：13:36:20]

[ok] - 2308 - 8165728
[尝试次数：4] - [剩余数量：2959] - [当前时间：13:36:20]

[ok] - 2308 - 8165748
[尝试次数：3] - [剩余数量：2958] - [当前时间：13:36:20]

[ok] - 2308 - 8165760
[尝试次数：2] - [剩余数量：2957] - [当前时间：13:36:20]

[ok] - 2308 - 5548884
[尝试次数：1] - [剩余数量：2956] - [当前时间：13:36:21]

[ok] - 2308 - 8165772
[尝试次数：1] - [剩余数量

[ok] - 2308 - 8918280
[尝试次数：2] - [剩余数量：2842] - [当前时间：13:36:58]

[ok] - 2308 - 8918328
[尝试次数：1] - [剩余数量：2841] - [当前时间：13:36:59]

[ok] - 2308 - 8918312
[尝试次数：7] - [剩余数量：2840] - [当前时间：13:36:59]

[ok] - 2308 - 8918288
[尝试次数：12] - [剩余数量：2839] - [当前时间：13:36:59]

[ok] - 2308 - 8918324
[尝试次数：4] - [剩余数量：2838] - [当前时间：13:36:59]

[ok] - 2308 - 8918336
[尝试次数：3] - [剩余数量：2837] - [当前时间：13:37:00]

[ok] - 2308 - 8918320
[尝试次数：1] - [剩余数量：2836] - [当前时间：13:37:00]

[ok] - 2308 - 8918356
[尝试次数：1] - [剩余数量：2835] - [当前时间：13:37:00]

[ok] - 2308 - 8918300
[尝试次数：14] - [剩余数量：2834] - [当前时间：13:37:01]

[ok] - 2308 - 8918340
[尝试次数：7] - [剩余数量：2833] - [当前时间：13:37:01]

[ok] - 2308 - 8165844
[尝试次数：4] - [剩余数量：2832] - [当前时间：13:37:02]

[ok] - 2308 - 8918352
[尝试次数：7] - [剩余数量：2831] - [当前时间：13:37:02]

[ok] - 2308 - 8918348
[尝试次数：7] - [剩余数量：2830] - [当前时间：13:37:02]

[ok] - 2308 - 8918344
[尝试次数：8] - [剩余数量：2829] - [当前时间：13:37:02]

[ok] - 2308 - 8165852
[尝试次数：5] - [剩余数量：2828] - [当前时间：13:37:03]

[ok] - 2308 - 8165848
[尝试次数：6] - [剩余数量

[ok] - 2308 - 9452580
[尝试次数：14] - [剩余数量：2713] - [当前时间：13:37:38]

[ok] - 2308 - 5548929
[尝试次数：7] - [剩余数量：2712] - [当前时间：13:37:39]

[ok] - 2308 - 8918592
[尝试次数：4] - [剩余数量：2711] - [当前时间：13:37:39]

[ok] - 2308 - 5662398
[尝试次数：4] - [剩余数量：2710] - [当前时间：13:37:39]

[ok] - 2308 - 9452572
[尝试次数：21] - [剩余数量：2709] - [当前时间：13:37:40]

[ok] - 2308 - 9452568
[尝试次数：23] - [剩余数量：2708] - [当前时间：13:37:40]

[ok] - 2308 - 5662416
[尝试次数：3] - [剩余数量：2707] - [当前时间：13:37:41]

[ok] - 2308 - 8918576
[尝试次数：14] - [剩余数量：2706] - [当前时间：13:37:41]

[ok] - 2308 - 5548935
[尝试次数：4] - [剩余数量：2705] - [当前时间：13:37:41]

[ok] - 2308 - 5662419
[尝试次数：4] - [剩余数量：2704] - [当前时间：13:37:41]

[ok] - 2308 - 5662413
[尝试次数：5] - [剩余数量：2703] - [当前时间：13:37:42]

[ok] - 2308 - 5662407
[尝试次数：5] - [剩余数量：2702] - [当前时间：13:37:42]

[ok] - 2308 - 5662428
[尝试次数：1] - [剩余数量：2701] - [当前时间：13:37:42]

[ok] - 2308 - 7104152
[尝试次数：2] - [剩余数量：2700] - [当前时间：13:37:43]

[ok] - 2308 - 5662422
[尝试次数：6] - [剩余数量：2699] - [当前时间：13:37:43]

[ok] - 2308 - 7104148
[尝试次数：1] - [剩余

[ok] - 2308 - 6690504
[尝试次数：2] - [剩余数量：2585] - [当前时间：13:38:20]

[ok] - 2308 - 6690496
[尝试次数：5] - [剩余数量：2584] - [当前时间：13:38:21]

[ok] - 2308 - 6690500
[尝试次数：3] - [剩余数量：2583] - [当前时间：13:38:21]

[ok] - 2308 - 8166188
[尝试次数：20] - [剩余数量：2582] - [当前时间：13:38:21]

[ok] - 2308 - 8382004
[尝试次数：2] - [剩余数量：2581] - [当前时间：13:38:22]

[ok] - 2308 - 6690508
[尝试次数：6] - [剩余数量：2580] - [当前时间：13:38:22]

[ok] - 2308 - 8166112
[尝试次数：1] - [剩余数量：2579] - [当前时间：13:38:22]

[ok] - 2308 - 8382008
[尝试次数：5] - [剩余数量：2578] - [当前时间：13:38:23]

[ok] - 2308 - 8918680
[尝试次数：1] - [剩余数量：2577] - [当前时间：13:38:24]

[ok] - 2308 - 8382000
[尝试次数：8] - [剩余数量：2576] - [当前时间：13:38:24]

[ok] - 2308 - 8918668
[尝试次数：6] - [剩余数量：2575] - [当前时间：13:38:24]

[ok] - 2308 - 8918672
[尝试次数：4] - [剩余数量：2574] - [当前时间：13:38:24]

[ok] - 2308 - 5548968
[尝试次数：1] - [剩余数量：2573] - [当前时间：13:38:25]

[ok] - 2308 - 5548965
[尝试次数：2] - [剩余数量：2572] - [当前时间：13:38:25]

[ok] - 2308 - 8918676
[尝试次数：5] - [剩余数量：2571] - [当前时间：13:38:26]

[ok] - 2308 - 8381996
[尝试次数：13] - [剩余数量

...

[ok] - 2308 - 12026077
[尝试次数：35] - [剩余数量：540] - [当前时间：13:49:03]

[ok] - 2308 - 12305825
[尝试次数：5] - [剩余数量：539] - [当前时间：13:49:03]

[ok] - 2308 - 12305869
[尝试次数：6] - [剩余数量：538] - [当前时间：13:49:03]

[ok] - 2308 - 12305897
[尝试次数：4] - [剩余数量：537] - [当前时间：13:49:04]

[ok] - 2308 - 12305913
[尝试次数：1] - [剩余数量：536] - [当前时间：13:49:04]

[ok] - 2308 - 12305885
[尝试次数：4] - [剩余数量：535] - [当前时间：13:49:04]

[ok] - 2308 - 12305881
[尝试次数：6] - [剩余数量：534] - [当前时间：13:49:04]

[ok] - 2308 - 12495893
[尝试次数：1] - [剩余数量：533] - [当前时间：13:49:04]

[ok] - 2308 - 12305901
[尝试次数：4] - [剩余数量：532] - [当前时间：13:49:05]

[ok] - 2308 - 12495905
[尝试次数：1] - [剩余数量：531] - [当前时间：13:49:05]

[ok] - 2308 - 12305905
[尝试次数：5] - [剩余数量：530] - [当前时间：13:49:05]

[ok] - 2308 - 12495901
[尝试次数：1] - [剩余数量：529] - [当前时间：13:49:05]

[ok] - 2308 - 12305893
[尝试次数：9] - [剩余数量：528] - [当前时间：13:49:05]

[ok] - 2308 - 12495909
[尝试次数：4] - [剩余数量：527] - [当前时间：13:49:06]

[ok] - 2308 - 12495949
[尝试次数：2] - [剩余数量：526] - [当前时间：13:49:07]

[ok] - 2308 - 12495953
[尝试次数：1] - [剩余数量

[ok] - 2308 - 12756029
[尝试次数：1] - [剩余数量：412] - [当前时间：13:49:42]

[ok] - 2308 - 12756013
[尝试次数：3] - [剩余数量：411] - [当前时间：13:49:43]

[ok] - 2308 - 12756105
[尝试次数：3] - [剩余数量：410] - [当前时间：13:49:43]

[ok] - 2308 - 12756125
[尝试次数：2] - [剩余数量：409] - [当前时间：13:49:43]

[ok] - 2308 - 12756069
[尝试次数：8] - [剩余数量：408] - [当前时间：13:49:43]

[ok] - 2308 - 12756113
[尝试次数：4] - [剩余数量：407] - [当前时间：13:49:44]

[ok] - 2308 - 12756085
[尝试次数：2] - [剩余数量：406] - [当前时间：13:49:44]

[ok] - 2308 - 12756101
[尝试次数：5] - [剩余数量：405] - [当前时间：13:49:44]

[ok] - 2308 - 12900657
[尝试次数：4] - [剩余数量：404] - [当前时间：13:49:44]

[ok] - 2308 - 12756077
[尝试次数：11] - [剩余数量：403] - [当前时间：13:49:45]

[ok] - 2308 - 12756141
[尝试次数：1] - [剩余数量：402] - [当前时间：13:49:45]

[ok] - 2308 - 12756081
[尝试次数：12] - [剩余数量：401] - [当前时间：13:49:45]

[ok] - 2308 - 12756065
[尝试次数：6] - [剩余数量：400] - [当前时间：13:49:45]

[ok] - 2308 - 12756137
[尝试次数：4] - [剩余数量：399] - [当前时间：13:49:46]

[ok] - 2308 - 12756133
[尝试次数：4] - [剩余数量：398] - [当前时间：13:49:46]

[ok] - 2308 - 12756189
[尝试次数：1] - [剩余数

[ok] - 2308 - 13702221
[尝试次数：7] - [剩余数量：282] - [当前时间：13:50:18]

[ok] - 2308 - 13702185
[尝试次数：13] - [剩余数量：281] - [当前时间：13:50:18]

[ok] - 2308 - 13702249
[尝试次数：1] - [剩余数量：280] - [当前时间：13:50:19]

[ok] - 2308 - 13702245
[尝试次数：2] - [剩余数量：279] - [当前时间：13:50:19]

[ok] - 2308 - 13702261
[尝试次数：2] - [剩余数量：278] - [当前时间：13:50:19]

[ok] - 2308 - 13702213
[尝试次数：8] - [剩余数量：277] - [当前时间：13:50:19]

[ok] - 2308 - 15535829
[尝试次数：4] - [剩余数量：276] - [当前时间：13:50:19]

[ok] - 2308 - 13702269
[尝试次数：2] - [剩余数量：275] - [当前时间：13:50:20]

[ok] - 2308 - 13702257
[尝试次数：1] - [剩余数量：274] - [当前时间：13:50:20]

[ok] - 2308 - 13702241
[尝试次数：8] - [剩余数量：273] - [当前时间：13:50:20]

[ok] - 2308 - 13702233
[尝试次数：8] - [剩余数量：272] - [当前时间：13:50:20]

[ok] - 2308 - 13702237
[尝试次数：9] - [剩余数量：271] - [当前时间：13:50:20]

[ok] - 2308 - 13702277
[尝试次数：3] - [剩余数量：270] - [当前时间：13:50:21]

[ok] - 2308 - 13702273
[尝试次数：5] - [剩余数量：269] - [当前时间：13:50:21]

[ok] - 2308 - 13702289
[尝试次数：1] - [剩余数量：268] - [当前时间：13:50:21]

[ok] - 2308 - 13702193
[尝试次数：9] - [剩余数量

[ok] - 2308 - 13961169
[尝试次数：4] - [剩余数量：154] - [当前时间：13:50:53]

[ok] - 2308 - 14705961
[尝试次数：1] - [剩余数量：153] - [当前时间：13:50:53]

[ok] - 2308 - 14706017
[尝试次数：2] - [剩余数量：152] - [当前时间：13:50:53]

[ok] - 2308 - 14705993
[尝试次数：5] - [剩余数量：151] - [当前时间：13:50:54]

[ok] - 2308 - 14706025
[尝试次数：2] - [剩余数量：150] - [当前时间：13:50:54]

[ok] - 2308 - 14706005
[尝试次数：4] - [剩余数量：149] - [当前时间：13:50:54]

[ok] - 2308 - 14706037
[尝试次数：1] - [剩余数量：148] - [当前时间：13:50:54]

[ok] - 2308 - 14706061
[尝试次数：3] - [剩余数量：147] - [当前时间：13:50:56]

[ok] - 2308 - 14705989
[尝试次数：9] - [剩余数量：146] - [当前时间：13:50:56]

[ok] - 2308 - 14706069
[尝试次数：1] - [剩余数量：145] - [当前时间：13:50:56]

[ok] - 2308 - 14706029
[尝试次数：6] - [剩余数量：144] - [当前时间：13:50:56]

[ok] - 2308 - 14706053
[尝试次数：2] - [剩余数量：143] - [当前时间：13:50:56]

[ok] - 2308 - 14706065
[尝试次数：5] - [剩余数量：142] - [当前时间：13:50:57]

[ok] - 2308 - 14706077
[尝试次数：1] - [剩余数量：141] - [当前时间：13:50:57]

[ok] - 2308 - 14706033
[尝试次数：7] - [剩余数量：140] - [当前时间：13:50:57]

[ok] - 2308 - 14706041
[尝试次数：10] - [剩余数量

[ok] - 2308 - 15535893
[尝试次数：5] - [剩余数量：25] - [当前时间：13:51:33]

[ok] - 2308 - 15359245
[尝试次数：5] - [剩余数量：24] - [当前时间：13:51:33]

[ok] - 2308 - 15359253
[尝试次数：4] - [剩余数量：23] - [当前时间：13:51:33]

[ok] - 2308 - 15359257
[尝试次数：4] - [剩余数量：22] - [当前时间：13:51:33]

[ok] - 2308 - 15535881
[尝试次数：7] - [剩余数量：21] - [当前时间：13:51:33]

[ok] - 2308 - 15359261
[尝试次数：3] - [剩余数量：20] - [当前时间：13:51:34]

[ok] - 2308 - 15359269
[尝试次数：1] - [剩余数量：19] - [当前时间：13:51:34]

[ok] - 2308 - 15359265
[尝试次数：4] - [剩余数量：18] - [当前时间：13:51:34]

[ok] - 2308 - 15535913
[尝试次数：1] - [剩余数量：17] - [当前时间：13:51:34]

[ok] - 2308 - 15359281
[尝试次数：1] - [剩余数量：16] - [当前时间：13:51:35]

[ok] - 2308 - 15359273
[尝试次数：3] - [剩余数量：15] - [当前时间：13:51:35]

[ok] - 2308 - 15535925
[尝试次数：1] - [剩余数量：14] - [当前时间：13:51:35]

[ok] - 2308 - 15535917
[尝试次数：4] - [剩余数量：13] - [当前时间：13:51:35]

[ok] - 2308 - 15359285
[尝试次数：2] - [剩余数量：12] - [当前时间：13:51:36]

[ok] - 2308 - 15535921
[尝试次数：4] - [剩余数量：11] - [当前时间：13:51:36]

[ok] - 2308 - 15535933
[尝试次数：2] - [剩余数量：10] - [当前时间：13: